<a href="https://colab.research.google.com/github/htapia/damju/blob/master/DAMJu_S2_L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semana 2 Libreta 2

## Primera aplicacion

Con el sistema de desarrollo listo, vamos a crear nuestra primera applicacion de forma interactiva. Al final podemos desplegarla en el dispositivo y ejecutarla ahi. En esta libreta realizaremos los siguietnes pasos:

1. Determinar 

# Fin de sesion.